In [3]:
import random
import matplotlib.pyplot as plt
import pylab
import numpy as np
from skimage.transform import rotate
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from scipy.misc import imsave
import os
import pickle
import shutil
from joblib import Parallel, delayed

#Rotation: RS140_x5_0000_gabor-blueNisslWide-segm-blueNisslRegular_spMaxDirAngle.npys
#RAW:   "/oasis/projects/nsf/csd181/yuncong/DavidData2014tif/RS140/x5/0000/RS140_x5_0000.tif"
#PATH: " /oasis/projects/nsf/csd181/yuncong/DavidData2014results/RS140/0000/"
#Example: "/oasis/projects/nsf/csd181/yuncong/DavidData2014v4/RS141/x5/0001/pipelineResults/RS141_x5_0001_segm-blueNissl_segmentation.npy"

In [4]:
SAMPLES = 29
IMG_PATHS = []
DATA_PATHS = []
PROC_PATHS = []
CODE_NAMES = []

for i in range(0, np.min((10, SAMPLES + 1))):
    val = "000" + str(i)
    IMG_PATHS.append("/Users/idanizhaki/Desktop/yuncong/DavidData2014tif/RS141/x5/" + val + "/RS141_x5_" + val + ".tif")
    DATA_PATHS.append("/Users/idanizhaki/Desktop/yuncong/DavidData2014results/RS141/" + val + "/")
    PROC_PATHS.append("RS141/x5/" + val)
    CODE_NAMES.append("blueNisslRegular")
    
for i in range(10, SAMPLES + 1):
    val = "00" + str(i)
    IMG_PATHS.append("/Users/idanizhaki/Desktop/yuncong/DavidData2014tif/RS141/x5/" + val + "/RS141_x5_" + val + ".tif")
    DATA_PATHS.append("/Users/idanizhaki/Desktop/yuncong/DavidData2014results/RS141/" + val + "/")
    PROC_PATHS.append("RS141/x5/" + val)
    CODE_NAMES.append("blueNisslRegular")
    
TARGET = "/Users/idanizhaki/Desktop/WebStem/photos/"

In [5]:
mapped = [0] * SAMPLES
props = [0] * SAMPLES
dists = [0] * SAMPLES
neighbors = [0] * SAMPLES
refs = {}

def worker(smpl, mapped):
    print str(smpl) + " of  " + str(SAMPLES)
    FNAMES = PROC_PATHS[smpl].replace("/", "_")
    FPATH = DATA_PATHS[smpl] 
    IMG_PATH = IMG_PATHS[smpl]
    PROC_PATH = PROC_PATHS[smpl]
    CODE_NAME = CODE_NAMES[smpl]
    
    # Load segments data
    SEGMENTS  = FPATH + FNAMES + "_segm-" + CODE_NAME + "_segmentation.npy" 
    ROTATIONS = FPATH + FNAMES + "_gabor-blueNisslWide-segm-" + CODE_NAME + "_spMaxDirAngle.npy" 
    NEIBRS         = FPATH + FNAMES + "_segm-" + CODE_NAME + "_neighbors.npy"
    TEXT_DIST   = FPATH + FNAMES + "_gabor-blueNisslWide-segm-" + CODE_NAME + "-vq-blueNissl_texHistPairwiseDist.npy" 
    TEXT_MAP   = FPATH + FNAMES + "_gabor-blueNisslWide-vq-blueNissl_texMap.npy"
    PROPS          = FPATH + FNAMES + "_segm-" + CODE_NAME + "_spProps.npy"
    data = np.load(SEGMENTS)
    N = data.max() + 1
    #rotData = np.zeros(N)
    rotData = np.load(ROTATIONS)
    nbrs = np.load(NEIBRS)
    texton_dist = np.load(TEXT_DIST)
    texton_map = np.load(TEXT_MAP)
    sp_props = np.load(PROPS)
    props[smpl] = sp_props
    dists[smpl] = texton_dist
    neighbors[smpl] = nbrs
    
    # Input image
    img = plt.imread(IMG_PATH)
    plt.imshow(img)
    refName = "src_" + str(smpl) + "_" + str(img.shape[0]) + "_" + str(img.shape[1]) +  "_0.jpg"
    refs[smpl] = refName
    imsave(TARGET + refName, img)
    
    # Collect images around a center (super-super-pixel)
    center_dists = pdist(sp_props[:, :2])
    center_dist_matrix = squareform(center_dists)
    k = 8
    k_neighbors = np.argsort(center_dist_matrix, axis=1)[:, 1:k+1]
    
    # Function to get the dimension of a super-super-pixel
    def getXYwithNbrs(v, trsh):
        minY, minX, maxY, maxX = sp_props[v, 4:]

        for o in k_neighbors[v]:
            if texton_dist[v, o] > trsh:
                return True, 0, 0, 0, 0

            minoY, minoX, maxoY, maxoX = sp_props[o, 4:]

            minX = min(minX, minoX)
            maxX = max(maxX, maxoX)
            minY = min(minY, minoY)
            maxY = max(maxY, maxoY)

        return False, minX, maxX, minY, maxY
    
    # Generate photos of super-super-pixel
    lenX, lenY = len(img), len(img[0])
    mapped[smpl] = {}
    #T = 0.7
    #T = 1000
    T = 0.8
    Txtns = np.max(texton_map) + 1

    for v in range(0, N):
        if v == -1: 
            continue

        # Mark all values equal to v
        b, minX, maxX, minY, maxY = getXYwithNbrs(v, T)
        if b:
            continue

        # Crop patch
        degree = np.degrees(rotData[v])
        H = maxX - minX
        W = maxY - minY
        buffX = H / 2 #np.min((np.max((20, minX - 1)), lenX - maxX - 1))
        buffY = W / 2 #np.min((np.max((20, minY - 1)), lenY - maxY - 1))
        patch = img[minY - buffY: maxY + buffY, minX - buffX : maxX + buffX]
        #patch2 = img[minX : maxX, minY : maxY]
        #imshow(patch2)
        #break

        # Invalid images (borders, etc.)
        if patch.shape[0] == 0 or patch.shape[1] == 0:
            continue
            
        # Temporary
        #degree = random.randint(0, 17) * 10
        patch = rotate(patch, degree, True)
        #imshow(patch)

        nCY, nCX, _ = patch.shape
        P = np.max((W, H))
        patch = patch[(nCY - P) / 2 : (nCY + P) / 2, (nCX - P) / 2 : (nCX + P) / 2]
        nCY, nCX, _ = patch.shape
        if nCY / nCX > 4 or nCX / nCY > 4:
            continue

        radius = int(np.round(np.min((nCY, nCX)) / 2))
        pointX = int(np.round(maxX - H / 2))
        pointY = int(np.round(maxY - W / 2))

        textons = [0] * Txtns
        '''for r in range(0, radius + 1):
            for x in range(0, radius + 1):
                for y in range(0, radius + 1):
                    if x ** 2 + y ** 2 == r ** 2:
                        rx = minX + x
                        ry = minY + y
                        val = texton_map[ry, rx]
                        if val >= 0:
                            textons[val] += 1'''

        for x in range(int(minX), int(maxX)):
            for y in range(int(minY), int(maxY)):
                val = texton_map[y, x]
                if val >= 0:
                    textons[val] += 1

        suffixName = "ref_" + str(smpl) + "_" + str(v) + "_" + str(pointX) + "_" + str(pointY) + "_" + str(radius)
        imgName = TARGET + suffixName
        mapped[smpl][v] = suffixName
        plt.imsave(imgName + ".png", patch)
        plt.clf();
        plt.cla();
        
        plt.hist(range(Txtns), bins=Txtns, weights=textons, normed = True, label=None)
        plt.axes().get_xaxis().set_visible(False)
        plt.axes().get_yaxis().set_visible(False)
        plt.savefig(imgName + '_hist.png')
        plt.clf();
        plt.cla();
        
    return mapped, props, dists, refs

In [247]:
restoreFromFile = True
if restoreFromFile:
    from os import listdir
    from os.path import isfile, join

    mypath = TARGET
    onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
    mapped = {}
    refs = {}

    for f in onlyfiles:
        if f.find("src_") == 0:
            cdata = f.split("_")
            refs[int(cdata[1])] = f
        elif f.find("ref_") == 0 and f.find("_hist") < 0:
            cdata = f.split("_")
            l1 = int(cdata[1])
            l2 = int(cdata[2])
            if l1 not in mapped:
                mapped[l1] = {}

            mapped[l1][l2] = f.split(".png")[0]

    props = [0] * SAMPLES
    dists = [0] * SAMPLES
    neighbors = [0] * SAMPLES

    for smpl in range(SAMPLES):
        #print str(smpl) + " of  " + str(SAMPLES)
        FNAMES = PROC_PATHS[smpl].replace("/", "_")
        FPATH = DATA_PATHS[smpl] 
        IMG_PATH = IMG_PATHS[smpl]
        PROC_PATH = PROC_PATHS[smpl]
        CODE_NAME = CODE_NAMES[smpl]

        # Load segments data
        NEIBRS         = FPATH + FNAMES + "_segm-" + CODE_NAME + "_neighbors.npy"
        TEXT_DIST   = FPATH + FNAMES + "_gabor-blueNisslWide-segm-" + CODE_NAME + "-vq-blueNissl_texHistPairwiseDist.npy" 
        PROPS          = FPATH + FNAMES + "_segm-" + CODE_NAME + "_spProps.npy"

        nbrs = np.load(NEIBRS)
        texton_dist = np.load(TEXT_DIST)
        sp_props = np.load(PROPS)
        props[smpl] = sp_props
        dists[smpl] = texton_dist
        neighbors[smpl] = nbrs  
        
else:
    res = Parallel(n_jobs=1)(delayed(worker)(i, mapped) for i in range(1))
    mapped2, props2, dists2, refs2 = map(np.array, zip(*res))

    mapped = [[]] * SAMPLES
    props = [[]] * SAMPLES
    dists = [[]] * SAMPLES
    refs = {}

    for i in range(0, SAMPLES):
        mapped[i] = mapped2[i][i]
        props[i] = props2[i][i]
        dists[i] = dists2[i][i]
        refs.update(refs2[i])
        
    import subprocess

    for r in refs:
        subprocess.call(["convert", TARGET + refs[r], "-resize", "50%", TARGET + refs[r]]);

## Create Cover

In [248]:
# save the map in case of a crash
#file = open("images/v2name.pck", "w")
#pickle.dump(mapped, file)
#file.close()
print "done"

done


In [249]:
# load the map in case of a crash
#file  = open ("images/v2name.pck")
#mapped = pickle.load(file)
#file.close();
print "done"

done


In [250]:
layers = []

'''# 0 -> 1 : 6
layers.append([0, [1324, 1325, 1326, 1327, 1328, 1264, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1430, 1431, 
                             1432, 1433, 1434, 1435, 1436, 1437, 1438, 1439, 1440, 1441, 1442, 1443, 1444, 1548, 1549, 1550, 1551, 1552, 1553],
                         1, [2030, 2032, 2033, 2034, 2035, 2120, 2121, 2122, 2123, 2124, 2215, 2216, 2217, 2218, 2182, 2302, 2304, 2305, 
                              2306, 2307, 2308, 2258, 2252, 2392, 2394, 2395, 2363, 2354,2478, 2479, 2480, 2501, 2560, 2561, 2562, 2667, 2626, 2645],
                         6]);

# 0 -> 1 : 7
layers.append([0, [3134, 3190, 3147, 3148, 3191, 3192, 3202, 3117, 3128, 3135, 3208, 3193, 3238, 3284, 3285, 3239, 3240,
                             3286, 3287, 3241, 3242, 3243, 3303, 3288, 3244, 3371, 3372, 3374, 3375, 3376, 3377, 3378, 3332, 3379, 
                             3391, 3380, 3381, 3482, 3435, 3465, 3466, 3467, 3468, 3436, 3437, 3574, 3545, 3546, 3547, 3548, 3549,
                             3646, 3628, 3629, 3630, 3631, 3711, 3676, 3670, 3679, 3712],
                        1, [2835, 2838, 2863, 2822, 2759, 2886, 2836, 2830, 2851, 2839, 2840, 2927, 2928, 2947, 2901, 2891, 2905, 2906, 2998, 2960],
                        7]);

# 0 -> 1 : 12
layers.append([0, [2130, 2139, 2131, 2079, 2021, 1968, 1907, 1921, 1928, 2132, 2133, 2080, 2055, 2249, 2250, 2242, 2243,
                              2244, 2181, 2251, 2366, 2367, 2296, 2351, 2355, 2297, 2356, 2472, 2467, 2468, 2410, 2411, 2469, 2524],
                        1, [2373, 2338, 2339, 2411, 2422, 2537, 2358, 2359, 2457],
                        12]);

# 0 -> 1 : 14
layers.append([0, [3057, 3115, 3210, 3140, 3211, 3214, 3302, 3226, 3306, 3313, 3316, 3385, 3402, 3405, 3415, 3426, 3439, 
                             3531, 3502, 3532, 3553, 3510, 3593, 3600, 3618, 3601, 3686, 3695, 3687, 3742],
                        1, [2552, 2553, 2554, 2555, 2638, 2639, 2640, 2641, 2706, 2720, 2721, 2722, 2778, 2795, 2796, 2797, 2798, 2837, 
                            2866, 2867, 2868, 2908, 2915, 2932, 2933, 2982, 2983, 2984, 2971, 3035, 3036, 3037, 3038, 3015, 3039, 3077, 
                            3085, 3086, 3087, 3088, 3078, 3079, 3117, 3127, 3128, 3129, 3130, 3131, 3132],
                        14]);

# 1 -> 2: 14
layers.append([1, [2093, 2115, 2116, 2210, 2171, 2211, 2212, 2297, 2298, 2299, 2327, 2385, 2386, 2387, 2412, 2475, 2471, 2472, 2470, 2533, 2534, 2535, 2642], 
                         2, [677, 631, 610, 676, 709, 737, 710, 782, 711, 691, 758, 712, 792, 826, 827, 848, 828, 781, 829, 830, 793, 908, 865, 875, 909, 876, 
                              910, 911, 912, 913, 993, 956, 994, 995, 996, 997, 998, 999, 1000, 1063, 1064, 1065, 1066, 1067, 1099, 1100, 1197, 1198, 1199, 
                              1144, 1164, 1303, 1265, 1304, 1266, 1253, 1414, 1415], 
                        14]);

# 2 -> 3: 10
layers.append([2, [3214, 3187, 3237, 3273, 3282, 3243, 3274, 3362, 3316, 3348, 3349, 3325, 3350, 3332, 3426, 3378, 3422, 3425, 3391, 3411, 3434, 3482, 3478, 3452, 3522], 
                         3, [342, 309, 359, 360, 376, 424, 377, 427, 448, 453, 446, 501, 442, 533, 521, 525, 534, 526, 605, 648, 617, 648, 699, 742, 706, 829, 830, 800, 934, 935], 
                         10]);

# 2 -> 3: 11
layers.append([2, [550, 575, 630, 677, 631, 610, 676, 709, 737, 710, 762, 711, 691, 758, 712, 792, 826, 827, 848, 828, 781, 829, 830, 907, 908,
                             865, 875, 909, 876, 910, 911, 912, 913, 956, 994, 995, 996, 997, 998, 999, 1000, 1001, 1063, 1064, 1065, 1066, 1067, 1099, 1100,
                             1198, 1199, 1144, 1164, 1200, 1201, 1304, 1266, 1253, 1305, 1306, 1414, 1415, 1344, 1372, 1515, 1466, 1516, 1625, 1626, 1627, 1732], 
                         3, [78, 74, 80, 134, 111, 116, 184, 166, 158, 155, 213, 235, 206, 220, 214, 201, 262, 265, 258, 249, 342, 309, 264, 305, 359, 360], 
                         11]);

# 2 -> 3: 12
layers.append([2, [3222, 3210, 3166, 3223, 3288, 3292, 3244, 3289, 3301, 3245, 3251,
                             3355, 3356, 3357, 3361, 3317, 3333, 3366, 3326, 3366, 3318, 3424,
                             3415, 3416, 3395, 3396, 3427, 3397, 3417, 3438, 3472, 3473, 3474,
                             3475, 3476, 3526, 3527, 3507, 3508],
                         3, [1799, 1746, 1836, 1914, 1965, 1948, 2006, 1937, 2124, 2114,
                             2070, 2071, 2047, 2125, 2227, 2158, 2215, 2171, 2228, 2148, 2172,
                            2235, 2315, 2271, 2316, 2317, 2259, 2337, 2260, 2318, 2331, 2320,
                            2377, 2353, 2378, 2379, 2362, 2438, 2462, 2453, 2480, 2481, 2482],
                         12]);
 
# 2 -> 3: 16
layers.append([2, [3070, 3063, 3079, 3064, 3168, 3169, 3159, 3227, 3233, 3240,
                              3261, 3304, 3311, 3313, 3335, 3371, 3375, 3382, 3420, 3454, 
                              3455, 3440, 3456, 3495, 3491, 3501, 3513, 3538, 3546, 3550,
                              3572, 3586, 3584, 3613, 3587, 3588, 3605, 3589, 3583, 3590, 3610,
                              3620, 3650, 3618, 3643, 3626, 3635, 3633, 3636, 3637, 3652,
                              3679, 3657, 3674, 3673, 3670, 3668, 3686, 3697, 3696, 3693,
                              3689, 3582, 3594, 3598, 3621, 3638, 3644, 3654, 3683, 3669, 
                              3503, 3545, 3516, 3504, 3517, 3552, 3559, 3561, 3562, 3563,
                              3598, 3599, 3606, 3609, 3607, 3581, 3615, 3644, 3625, 3639, 
                              3645, 3627, 3619, 3649, 3622, 3658, 3665, 3678, 3659, 3655,
                              3685, 3682, 3680, 3661, 3661, 3646, 3666, 3681, 3653, 3684,
                              3667, 3690, 3675, 3698, 3662, 3687, 3698, 3688, 3701, 3694,
                              3702, 3695, 3703, 3691, 3700, 3039, 3013, 3014, 3015, 3016,
                              3017, 3103, 3104, 3105, 3106, 3107, 3170, 3199, 3200, 3201,
                              3202, 3247, 3264, 3281, 3282, 3320, 3329, 3339, 3330, 3401,
                              3402, 3403, 3404, 3448, 3461, 3462, 3463],
                        3, [356, 313, 314, 317, 324, 354, 362, 383, 417, 418, 419, 420, 426,
                             371, 369, 492, 493, 494, 495, 496, 497, 505, 498, 443, 508, 436,
                             445, 438, 499, 575, 576, 577, 578, 579, 580, 581, 582, 528, 591,
                             516, 583, 522, 588, 662, 663, 664, 665, 666, 667, 668, 625, 675,
                             603, 669, 608, 676, 697, 513, 509, 437, 450, 435, 434, 368, 367, 
                             365, 366, 593, 589, 520, 584, 517, 511, 439, 432, 422, 429, 423,
                             678, 679, 626, 594, 525, 590, 537, 585, 586, 587, 764, 768, 760,
                             863, 854, 855, 856, 857, 954, 955, 906, 907, 956, 957, 958, 959,
                             960, 961, 962, 963, 979, 1051, 1052, 1053, 1054, 1055, 1056,
                             1057, 1058, 1059, 1004, 1061, 1103, 1168, 1082, 1152, 1153, 1154,
                             1155, 1156, 1157, 1158, 1167, 1159, 1190, 1256, 1257],
                        16]);
'''

layers = pickle.load(open("/Users/idanizhaki/Desktop/WebStem/layers.pck", "rb"))

In [251]:
for i in range(len(layers)):
    layers[i][1] = [n for n in layers[i][1] if n in mapped[layers[i][0]]]
    layers[i][3] = [n for n in layers[i][3] if n in mapped[layers[i][2]]]
    random.shuffle(layers[i][1])
    random.shuffle(layers[i][3])

#for i in range(SAMPLES):
 #   for k in mmapper[i]:
  #      mmapper[i][k] = [n for n in mmapper[i][k] if n in mapped[i]]
        #random.shuffle(mmapper[i][k])

In [252]:
#print mmapper[2]

In [253]:
'''layers = [];

for i in range(SAMPLES - 1):
    for j in range(i + 1, SAMPLES):
        for k in mmapper[i]:
            if mmapper[i][k] and k in mmapper[j] and mmapper[j][k]:
                layers += [(i, mmapper[i][k], j, mmapper[j][k])]
                
                
print layers'''

'layers = [];\n\nfor i in range(SAMPLES - 1):\n    for j in range(i + 1, SAMPLES):\n        for k in mmapper[i]:\n            if mmapper[i][k] and k in mmapper[j] and mmapper[j][k]:\n                layers += [(i, mmapper[i][k], j, mmapper[j][k])]\n                \n                \nprint layers'

In [254]:
'''
lay0n14 = [n for n in lay0n14 if n in mapped[0]]
lay1n14 = [n for n in lay1n14 if n in mapped[1]]

lay0n6 = [n for n in lay0n6 if n in mapped[0]]
lay1n6 = [n for n in lay1n6 if n in mapped[1]]

print "Layer 14"
print lay0n14
print lay1n14

print "Layer 6"
print lay0n6
print lay1n6

layers = [(lay0n14, lay1n14), (lay0n6, lay1n6)]
'''

def overlaps(layer1, spxl1, layer2, spxl2):
    if layer1 == 0:
        return spxl1 in lay0n14 and spxl2 in lay1n14
    return spxl2 in lay0n14 and spxl1 in lay1n14

In [255]:
Td = 700  # Pixels distance
Th = 0.75  # Histogram distance

In [256]:
index = 1;
for idx in range(len(layers)):
    tuples = layers[idx];
    
    smpl1 = tuples[0];
    smpl2 = tuples[2];
    
    potential1 = tuples[1]
    toRemove1 = set()
    
    for i in range(len(potential1)):
        tuples = layers[idx];
        vI = potential1[i]
        
        centI = props[smpl1][vI][:2]
        #nbrsI = [x for x in potential1 if x != vI and numpy.linalg.norm(centI - props[smpl1][x][:2]) < Td and dists[smpl1][vI, x] < Th]
        nbrsI = [x for x in potential1 if x != vI and dists[smpl1][vI, x] < Th and np.linalg.norm(centI - props[smpl1][x][:2]) < Td]
        
        potential2 = tuples[3]
        toRemove2 = set()
        
        for j in range(len(potential2)):
            vJ = potential2[j]
            
            try:
                centJ = props[smpl2][vJ][:2]
            except:
                continue
            
            nbrsJ = [x for x in potential2 if x != vJ and dists[smpl2][vJ, x] < Th and np.linalg.norm(centI - props[smpl2][x][:2]) < Td]
        
            lenI = len(nbrsI)
            lenJ = len(nbrsJ)
            if lenI + lenJ < 32:
                continue            

            if (lenI < 7 or lenJ < 7):
                continue
            if (lenI < 16):
                nbrsJ = nbrsJ[: 16 + 16 - lenI]
            elif (lenJ < 16):
                nbrsI = nbrsI[: 16 + 16 - lenJ]
            else:
                nbrsI = nbrsI[: 16]
                nbrsJ = nbrsJ[: 16]
        
            
            folderName = TARGET + "group_" + str(index) + "/"
            shutil.rmtree(folderName, True)
            os.mkdir(folderName)

            toRemove1.add(vI)
            mFile = mapped[smpl1][vI]
            os.symlink("../" + mFile + ".png", folderName + "img_" + mFile + ".png")
            os.symlink("../" + mFile + "_hist.png", folderName + "hist_" + mFile  + ".png")

            toRemove2.add(vJ)
            mFile = mapped[smpl2][vJ]
            os.symlink("../" + mFile + ".png", folderName + "img_" + mFile + ".png")
            os.symlink("../" + mFile + "_hist.png", folderName + "hist_" + mFile  + ".png")
    
            for n in nbrsI:
                toRemove1.add(n)
                mFile = mapped[smpl1][n]
                os.symlink("../" + mFile + ".png", folderName + mFile + ".png")

            for n in nbrsJ:
                toRemove2.add(n)
                mFile = mapped[smpl2][n]
                os.symlink("../" + mFile + ".png", folderName + mFile + ".png")

            os.symlink("../" + refs[smpl1], folderName + "org_" + str(smpl1));
            os.symlink("../" + refs[smpl2], folderName + "org_" + str(smpl2));

            #print index
            print index
            index += 1 
            i -= 1
            break
            
        if toRemove2:
            potential2 = [v for v in potential2 if not v in toRemove2]
            layers[idx] = (smpl1, potential1, smpl2, potential2)
            
    if toRemove1:
        potential1 = [v for v in potential1 if not v in toRemove1]
        layers[idx] = (smpl1, potential1, smpl2, potential2)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [6]:
maxKey = max(mapped.keys())
potential = list(sort(mapped.keys()))

Pair14 = [3023, 3057, 3039, 3115, 3210, 3121, 3140, 3211, 3214, 3302, 3226, 3247, 3306, 3313, 3316, 3385, 3402, 3405, 3415, 3426, 3439, 3531, 3502, 3532, 3553, 3510, 3593, 3600, 3618, 3601, 3663, 3686, 3695, 3717, 3687, 3742]
Pair6 = [1230, 1275, 1323, 1324, 1325, 1326, 1327, 1328, 1264, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336,  1428, 1429, 1430, 1431, 1432, 1433, 1434, 1435, 1436, 1437, 1438, 1439, 1440, 1441, 1442, 1443, 1541, 1542, 1543, 1544, 1545, 1546, 1547, 1548, 1549, 1550, 1551, 1552, 1553, 1554, 1555, 1556]

keys14 = [key for key in Pair14 if key in mapped ]
keys6 = [key for key in Pair6 if key in mapped ]
        
print keys14
print keys6

rkeys14 = []
rkeys6 = []

for i in range(0, len(potential)):
    if potential[i] in keys14:
        rkeys14.append(i)
    if potential[i] in keys6:
        rkeys6.append(i)
        
Pairs = []
for k14 in rkeys14:
    for k6 in rkeys6:
        Pairs.append((k6, k14))
    
'''
for p in Pairs:
    if p[0] in potential:
        potential.remove(p[0])
    if p[1] in potential:    
        potential.remove(p[1])
        
print potential
'''

[]
[1230, 1335, 1434, 1542]


'\nfor p in Pairs:\n    if p[0] in potential:\n        potential.remove(p[0])\n    if p[1] in potential:    \n        potential.remove(p[1])\n        \nprint potential\n'

In [7]:
N = len(potential)
index = 1
T = 0.10
S = []
nS = set()

print Pairs

i = 0
for (i, j) in Pairs:
    vI = potential[i]
    centI = sp_props[vI][:2]
    chosen = [vI]
    
    vJ = potential[j]
    centJ = sp_props[vJ][:2]
    
    chosen += [vJ]        
        
    changed = False

    for k in range(1, len(potential)):
        if k == i or k == j:
            continue
        
        vK = potential[k]
        centK = sp_props[vK][:2]

        distIKd = np.sqrt(sum((centI - centK)**2))
        if distIKd < Td:
            distJKd = np.sqrt(sum((centJ - centK)**2))
            if distJKd < Td:
                continue

        distIKh = texton_dist[vI, vK]
        if distIKh > Th:
            distJKh = texton_dist[vJ, vK]
            if distJKh > Th:
                continue

        changed = True
        chosen += [vK]
        #if len(chosen) >= 34:
          #  break

    if not changed:
        continue

    i += 1
    if len(chosen) < 34:
        continue

    folderName = TARGET + "group_" + str(index) + "/"
    shutil.rmtree(folderName, True)
    os.mkdir(folderName)
    
    #print mapped
    for n in chosen:
        fle = mapped[n]
        #print folderName + fle
        potential.remove(n)
        os.symlink(fle, folderName + fle)
    
    #print chosen[0]
    #break
    index += 1    

'''
while potential:
    folderName = TARGET + "group_" + str(index) + "/"
    shutil.rmtree(folderName, True)

    candidate = potential.popitem()
    nS.add(candidate)
    
    iter = potential.iteritems()
    remove = set()
    
    
    while True:
        try:
            next = iter.next()
        except:
            break
        succ = True
        v1 = next[0]
        for k in nS:
            v2 = k[0]
          #  print v1, v2
            if texton_dist[v1, v2] > T:
                succ = False
                break
        
        if succ:
            remove.add(next[0])
            nS.add(next)
            
    for i in remove:
        del potential[i]
        
    if (len(nS) >= 8):
        os.mkdir(folderName)
        #os.mkdir(folderName + "images")
        for n in nS:
            print n[1], folderName + n[1]
            os.symlink(n[1], folderName + n[1])
        index = index + 1
        S.append(nS)
    nS = set()
    
for s in S:
    print s
    
'''

[]


'\nwhile potential:\n    folderName = TARGET + "group_" + str(index) + "/"\n    shutil.rmtree(folderName, True)\n\n    candidate = potential.popitem()\n    nS.add(candidate)\n    \n    iter = potential.iteritems()\n    remove = set()\n    \n    \n    while True:\n        try:\n            next = iter.next()\n        except:\n            break\n        succ = True\n        v1 = next[0]\n        for k in nS:\n            v2 = k[0]\n          #  print v1, v2\n            if texton_dist[v1, v2] > T:\n                succ = False\n                break\n        \n        if succ:\n            remove.add(next[0])\n            nS.add(next)\n            \n    for i in remove:\n        del potential[i]\n        \n    if (len(nS) >= 8):\n        os.mkdir(folderName)\n        #os.mkdir(folderName + "images")\n        for n in nS:\n            print n[1], folderName + n[1]\n            os.symlink(n[1], folderName + n[1])\n        index = index + 1\n        S.append(nS)\n    nS = set()\n    \nfor s

In [8]:
print "Done"

Done


In [ ]:


for smpl1 in range(0, SAMPLES - 1):
    maxKey1 = max(mapped[smpl1].keys())
    potential1 = list(sort(mapped[smpl1].keys()))
    index = 1
    count = 0
    
    for smpl2 in range(smpl1 + 1, SAMPLES):
        maxKey2 = max(mapped[smpl2].keys())
        potential2 = list(sort(mapped[smpl2].keys()))
        n1 = len(potential1)
        
        for i in range(n1):
            vI = potential1[i]
            centI = props[smpl1][vI][:2]
            nbrsI = [x for x in potential1 if x != i and numpy.linalg.norm(centI - props[smpl1][x][:2]) < Td and dists[smpl1][vI, x] < Th]
            
            chosen = [(smpl1, vI)]
            n2 = len(potential2)
                
            for j in range(n2):            
                #print n2, len(potential2)
                vJ = potential2[j]
                
                if not overlaps(smpl1, vI, smpl2, vJ):
                    continue
                    
                centJ = props[smpl2][vJ][:2]
                nbrsJ = [x for x in potential2 if x != j and numpy.linalg.norm(centJ - props[smpl2][x][:2]) < Td and dists[smpl1][vJ, x] < Th]
                
                if len(nbrsI) + len(nbrsJ) < 32:
                    continue

                print len(nbrsI), len(nbrsJ)
                fdsf
                
                
                # Check right distances
                distIKd = np.sqrt(sum((centI - centK)**2))
                if distIKd < Td:
                    distJKd = np.sqrt(sum((centJ - centK)**2))
                    if distJKd < Td:
                        continue

                distIKh = texton_dist[vI, vK]
                if distIKh > Th:
                    distJKh = texton_dist[vJ, vK]
                    if distJKh > Th:
                        continue
                
                #centJ = sp_props[smpl2][vJ][:2]

                chosen += [(smpl2, vJ)]
                if (len(chosen) >= 34):
                    break

            if len(chosen) < 34:
                continue

            folderName = TARGET + "group_" + str(index) + "/"
            shutil.rmtree(folderName, True)
            os.mkdir(folderName)
    
            #print mapped
            for (k, n) in chosen:
                mFile = mapped[k][n]
                #print folderName + mFile
                if k == smpl1:
                    print k
                    #potential1.remove(n)
                else:
                    potential2.remove(n)
                    
                if count < 2:
                    os.symlink("../" + mFile + ".png", folderName + "img_" + mFile + ".png")
                    os.symlink("../" + mFile + "_hist.png", folderName + "hist_" + mFile  + ".png")
                else:
                    os.symlink("../" + mFile + ".png", folderName + mFile + ".png")
                count += 1
            
            os.symlink("../" + refs[smpl1], folderName + "org_" + str(smpl1));
            os.symlink("../" + refs[smpl2], folderName + "org_" + str(smpl2));
    
            #print chosen[0]
            #break
            index += 1    
            count = 0